In [1]:
# Standard Library Imports (if any)

# Third-party Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from IPython.display import display, HTML
from adjustText import adjust_text

# Local Module Imports
from utils import *
from data_handling import *
from plot import *


In [ ]:

dm = DataManager(
    raw_dir = "../data/raw",
    save_dir = "../data/clean"
)
# dm.clean_data()
# dm.save_data()
dm.filter_data(["BE", "US", "ES"], (2002,2004))

In [4]:
# Access datasets
cpis = dm.get_dataset("cpis")
ds = dm.get_dataset("ds")
fed = dm.get_dataset("fed")
wb = dm.get_dataset("wb")
wfe = dm.get_dataset("wfe")
gdp = dm.get_dataset("gdp")

In [11]:
gdp

,2002,2003,2004
Country,,,
BE,2.583836e+11,3.180825e+11,3.692147e+11
ES,7.089382e+11,9.079632e+11,1.069829e+12
US,1.092911e+13,1.145645e+13,1.221720e+13


In [ ]:
countries = DS.CODES
cpis = cpis.get_data(issuers="WR", holders=countries, periods=range(2001,2005)).droplevel(level="Counterpart Country")
gdp = gdp.iloc[:,2:6]
wb = wb.loc[DS.CODES, 2001:2004]

AttributeError: 'DataFrame' object has no attribute 'get_data'

In [ ]:
y = (cpis/gdp).mean(axis=1)
X_1 = (gdp/wb).mean(axis=1)
X_2 = cpis.calculate_excess_returns_matrix(fed, ds, (1990,2023), True).mean(axis=1)
X_3 = cpis.calculate_excess_returns_matrix(fed, ds, (2001,2004), True).var(axis=1)
X = pd.concat([X_1, X_2, X_3], axis=1, keys=["gdp/cap", "mean", "variance"])
X = sm.add_constant(X)
model = sm.OLS(y,X).fit(cov_type="HAC", cov_kwds={"maxlags": 4})
print(model.summary())

In [ ]:
pd.concat([X,y], axis=1)

In [ ]:
plt.scatter(X["mean"], y)

In [ ]:
cpis